<a href="https://colab.research.google.com/github/arny617/MLNanodegree/blob/master/Crypto_SentimentAnalysis_RandomForest_UsingTrigrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import numpy as np
import nltk
import os
nltk.download('wordnet')
import pandas as pd
import numpy as np
import pickle
import joblib

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import gensim

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
!pip install OpenBlender

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 245kB 6.2MB/s 
  Created wheel for OpenBlender: filename=OpenBlender-2.6-cp36-none-any.whl size=6608 sha256=ba7e43f0e734ca75d36b3f0c41df8d8131d7ea94ceb60360d8583d271b38e7e2
  Stored in directory: /root/.cache/pip/wheels/fd/29/7b/377afef16a792ed174a9ade5a6707ef87843e214476d0c8205
Successfully built OpenBlender


In [ ]:
import OpenBlender

In [7]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
ROOT = "/content/gdrive/My Drive/Colab Notebooks/NLP/Fast.ai_IntroToNLP/"
os.chdir(ROOT)

Mounted at /content/gdrive


### Get Data using OpenBlender

In [ ]:
api_token = '5fc5ad47951629617230bee5MDHzrVvlWhz1MZm92QeQqc5DrybPBT'

In [ ]:
action = 'API_getObservationsFromDataset'# ANCHOR: 'Bitcoin vs USD'
  
parameters = { 
    'token' : api_token,
    'id_dataset' : '5d4c3af79516290b01c83f51',
    'date_filter':{"start_date" : "2020-01-01",
                   "end_date" : "2020-08-29"} }
df = pd.read_json(json.dumps(OpenBlender.call(action, parameters)['sample']), convert_dates=False, convert_axes=False).sort_values('timestamp', ascending=False)
df.reset_index(drop=True, inplace=True)
df['date'] = [OpenBlender.unixToDate(ts, timezone = 'GMT') for ts in df.timestamp]
df = df.drop('timestamp', axis = 1)

Task ID: '5fc84fb90895fafb4a9d8d4b'.
Total estimated consumption: 5832.61 processing units.
Task confirmed. Starting download..
100.0 % completed.


In [ ]:
df['log_ret'] = np.log(df['price']) - np.log(df['open'])

In [ ]:
df['target'] = [1 if log_diff > 0 else 0 for log_diff in df['log_ret']]

In [ ]:
format = '%d-%m-%Y %H:%M:%S'
timezone = 'GMT'
df['u_timestamp'] = OpenBlender.dateToUnix(df['date'], 
                                           date_format = format, 
                                           timezone = timezone)
df = df[['date', 'u_timestamp', 'price', 'target']]
df.head()

,date,u_timestamp,price,target
0,28-08-2020 17:00:00,1.598634e+09,11545.0,1
1,27-08-2020 17:00:00,1.598548e+09,11343.0,0
2,26-08-2020 17:00:00,1.598461e+09,11471.0,1
3,25-08-2020 17:00:00,1.598375e+09,11341.0,0
4,23-08-2020 17:00:00,1.598202e+09,11663.0,0


In [ ]:
blend_source = {
                'id_dataset':'5ea2039095162936337156c9',
                'feature' : 'text'
            }
# Now, let's 'timeBlend' it to our dataset
df_blend = OpenBlender.timeBlend( token = api_token,
                                  anchor_ts = df.u_timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
df = pd.concat([df, df_blend.loc[:, df_blend.columns != 'u_timestamp']], axis = 1)
df.head()

Task ID: '5fc850470895fafb4a9d8d4c'.
Total estimated consumption: 9440.0 processing units.
Task confirmed. Starting download..
100%


,date,u_timestamp,price,target,timestamp,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days
0,28-08-2020 17:00:00,1.598634e+09,11545.0,1,1577898000,3,[etoro ceo yoni assia on reaching 12 million u...
1,27-08-2020 17:00:00,1.598548e+09,11343.0,0,1577984400,7,[government confirms crypto profits not taxabl...
2,26-08-2020 17:00:00,1.598461e+09,11471.0,1,1578070800,8,[ticker tool uses the bch blockchain to provid...
3,25-08-2020 17:00:00,1.598375e+09,11341.0,0,1578157200,6,[crypto fundraising is changing again in 2020 ...
4,23-08-2020 17:00:00,1.598202e+09,11663.0,0,1578243600,5,[subhash chandra garg on the future of crypto ...


In [ ]:
df['text'] = df['BITCOIN_NE.text_last1days'].apply(lambda x:"xxeos ".join(x))

In [ ]:
vader_analyzer = SentimentIntensityAnalyzer()

In [ ]:
df["text_polarity"] = df["text"].apply(lambda x : vader_analyzer.polarity_scores(x))
df["text_subjectivity"] = df["text"].apply(lambda x : TextBlob(x).sentiment[1])

In [ ]:
# BTC Text Vectorizer
blend_source = { 
                'id_textVectorizer':'5f739fe7951629649472e167'
               }
df_blend = OpenBlender.timeBlend( token = api_token,
                                  anchor_ts = df.u_timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw').add_prefix('VEC.')

Task ID: '5fc857840895fafb4a9d8d4c'.
Total estimated consumption: 10479.43 processing units.
Task confirmed. Starting download..
100%


In [ ]:
df_anchor = pd.concat([df, df_blend.loc[:, df_blend.columns != 'timestamp']], axis = 1)
df_anchor.head()

,date,u_timestamp,price,target,timestamp,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days,text,text_polarity,text_subjectivity,VEC.block reward,VEC.shop,VEC.code,VEC.virgin,VEC.chain,VEC.miner,VEC.demand,VEC.dollar,VEC.month,VEC.funding debate,VEC.kicks,VEC.httpbitcoincom wallet,VEC.battle,VEC.chair,VEC.issues,VEC.chinese,VEC.electricity,VEC.billion loans,VEC.defi flash,VEC.group,VEC.countdown,VEC.list,VEC.charge,VEC.program,VEC.stimulus checks,VEC.helps,VEC.encrypted,VEC.breach,VEC.lawmakers,VEC.worth,...,VEC.coinjoin,VEC.star,VEC.coindeal,VEC.congress,VEC.bitcoin mining,VEC.grayscale,VEC.printing,VEC.return,VEC.regulations,VEC.crypto traders,VEC.global economy,VEC.allowing,VEC.ceos,VEC.authorities,VEC.funding,VEC.plustoken,VEC.stocks,VEC.presidential,VEC.pool,VEC.billion,VEC.sanctions,VEC.bch,VEC.assets,VEC.calls,VEC.tank,VEC.space,VEC.youtube,VEC.daily,VEC.reports,VEC.2000,VEC.safe,VEC.heats,VEC.profits,VEC.volumes,VEC.time,VEC.laws,VEC.expands,VEC.integrates,VEC.digital gold,VEC.minister
0,28-08-2020 17:00:00,1.598634e+09,11545.0,1,1577898000,3,[etoro ceo yoni assia on reaching 12 million u...,etoro ceo yoni assia on reaching 12 million us...,"{'neg': 0.0, 'neu': 0.954, 'pos': 0.046, 'comp...",0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,27-08-2020 17:00:00,1.598548e+09,11343.0,0,1577984400,7,[government confirms crypto profits not taxabl...,government confirms crypto profits not taxable...,"{'neg': 0.032, 'neu': 0.787, 'pos': 0.181, 'co...",0.462500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,26-08-2020 17:00:00,1.598461e+09,11471.0,1,1578070800,8,[ticker tool uses the bch blockchain to provid...,ticker tool uses the bch blockchain to provide...,"{'neg': 0.122, 'neu': 0.785, 'pos': 0.093, 'co...",0.254167,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,25-08-2020 17:00:00,1.598375e+09,11341.0,0,1578157200,6,[crypto fundraising is changing again in 2020 ...,crypto fundraising is changing again in 2020 x...,"{'neg': 0.063, 'neu': 0.847, 'pos': 0.09, 'com...",0.518182,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,23-08-2020 17:00:00,1.598202e+09,11663.0,0,1578243600,5,[subhash chandra garg on the future of crypto ...,subhash chandra garg on the future of crypto x...,"{'neg': 0.049, 'neu': 0.772, 'pos': 0.179, 'co...",0.319886,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_anchor.to_csv("BTC_Final.csv")

### Load Pre Saved Data

In [9]:
df_anchor = pd.read_csv("BTC_Final.csv",usecols=[1,2,3,4,5,6,7,8,9,10])

In [10]:
df_anchor.head()

,date,u_timestamp,price,target,timestamp,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days,text,text_polarity,text_subjectivity
0,28-08-2020 17:00:00,1.598634e+09,11545.0,1,1577898000,3,['etoro ceo yoni assia on reaching 12 million ...,etoro ceo yoni assia on reaching 12 million us...,"{'neg': 0.0, 'neu': 0.954, 'pos': 0.046, 'comp...",0.000000
1,27-08-2020 17:00:00,1.598548e+09,11343.0,0,1577984400,7,['government confirms crypto profits not taxab...,government confirms crypto profits not taxable...,"{'neg': 0.032, 'neu': 0.787, 'pos': 0.181, 'co...",0.462500
2,26-08-2020 17:00:00,1.598461e+09,11471.0,1,1578070800,8,['ticker tool uses the bch blockchain to provi...,ticker tool uses the bch blockchain to provide...,"{'neg': 0.122, 'neu': 0.785, 'pos': 0.093, 'co...",0.254167
3,25-08-2020 17:00:00,1.598375e+09,11341.0,0,1578157200,6,['crypto fundraising is changing again in 2020...,crypto fundraising is changing again in 2020 x...,"{'neg': 0.063, 'neu': 0.847, 'pos': 0.09, 'com...",0.518182
4,23-08-2020 17:00:00,1.598202e+09,11663.0,0,1578243600,5,['subhash chandra garg on the future of crypto...,subhash chandra garg on the future of crypto x...,"{'neg': 0.049, 'neu': 0.772, 'pos': 0.179, 'co...",0.319886


In [ ]:
#df_anchor["text_polarity"] = df_anchor["text_polarity"].astype('object')

In [11]:
from ast import literal_eval

In [12]:
def try_literal_eval(s):
    try:
        return literal_eval(s)
    except ValueError:
        return s

In [13]:
df_anchor["text_polarity"] = df_anchor["text_polarity"].apply(lambda x:try_literal_eval(x))

In [15]:
df_anchor["text_posneg"] = df_anchor["text_polarity"].apply(lambda x : 0 if x["neg"]==0.0 else x["pos"]/x["neg"]) 


In [16]:
df_anchor["text_polarity"] = df_anchor["text_polarity"].apply(lambda x : x["compound"]) 

In [17]:
import nltk
from nltk.corpus import stopwords as sw
from nltk.stem import WordNetLemmatizer 
from nltk.stem import SnowballStemmer
import gensim
from gensim.utils import simple_preprocess
from tqdm import tqdm, tqdm_notebook
from tqdm import notebook

In [18]:
def clean_text(text_blob):
    """
    This function aims to perform a series of text processing tasks - 
    For each paragraph
        a)basic pre-processing and removal of punctuations
        b)perform POS tagiing
        c)remove stopwords, non alphabetical character sequences and proper nouns
    
    Input : Simple raw text corpus
    Output : list of processed text
    """
    stopwords = sw.words("english")
    ps = SnowballStemmer("english")
    #ps = PorterStemmer()
    para = ' '.join(simple_preprocess(text_blob,deacc = True)) #Basic preprocessing and removal of punctuations
    tagged_list = nltk.tag.pos_tag(para.split())
    fin_text_blob = []
    for word,tag in tagged_list:
        if((word.lower() not in stopwords) and (word.isalpha()) and (len(word) >= 3) and (tag != "NNP" and tag != "NNPS")):
            fin_text_blob.extend([ps.stem(word.lower())])
    return(fin_text_blob) #This function hence returns the original text blob, processed 

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
text_list = []
for i in notebook.tqdm(range(df_anchor.shape[0])):
    text_blob = clean_text(str(df_anchor.text[i]))
    text_list.extend([text_blob])

In [ ]:
#flat_list = [item for sublist in text_list for item in sublist]

In [22]:
import gensim
bigram = gensim.models.Phrases(text_list, min_count=5, threshold=50) #Generating bigrams
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [23]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [24]:
data_words_bigrams = make_bigrams(text_list) 

In [25]:
trigram = gensim.models.Phrases(data_words_bigrams, min_count=5, threshold=50)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [26]:
def make_trigrams(texts):
    return [trigram_mod[doc] for doc in texts]

In [27]:
data_words_trigrams = make_trigrams(data_words_bigrams) 

In [28]:
df_anchor['text_new'] = [" ".join(item) for item in data_words_trigrams]

In [31]:
count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(df_anchor['text_new'].values.astype('U'))

In [32]:
doc_term_matrix

<236x1312 sparse matrix of type '<class 'numpy.int64'>'
	with 8141 stored elements in Compressed Sparse Row format>

In [33]:
features = count_vect.get_feature_names()
features = list(map('VEC_'.__add__,features))

In [34]:
df_res = pd.DataFrame(doc_term_matrix.toarray(), columns=features)

In [35]:
df_res.head()

,VEC_abc,VEC_acceler,VEC_accept,VEC_access,VEC_accord,VEC_account,VEC_accus,VEC_acquir,VEC_acquisit,VEC_act,VEC_action,VEC_activ,VEC_ad,VEC_adam,VEC_add,VEC_address,VEC_adopt,VEC_advertis,VEC_affect,VEC_africa,VEC_african,VEC_agenc,VEC_agent,VEC_aggreg,VEC_ahead,VEC_aid,VEC_aim,VEC_air,VEC_airdrop,VEC_airport,VEC_algorithm,VEC_alleg,VEC_allow,VEC_alltim,VEC_altcoin,VEC_altern,VEC_america,VEC_american,VEC_amid,VEC_aml,...,VEC_watchdog,VEC_wave,VEC_way,VEC_wealth,VEC_web,VEC_websit,VEC_week,VEC_wendi,VEC_west,VEC_whale,VEC_white,VEC_wild,VEC_win,VEC_wirecard,VEC_wirex,VEC_withdraw,VEC_withdrawn,VEC_woman,VEC_wordpress,VEC_world,VEC_worldwid,VEC_worri,VEC_worst,VEC_worth,VEC_wrap,VEC_xrp,VEC_yam,VEC_year,VEC_yearold,VEC_yield,VEC_york,VEC_young,VEC_youtub,VEC_yuan,VEC_zcash,VEC_zebpay,VEC_zec,VEC_zero,VEC_zhuoer,VEC_zone
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
df_new = df_anchor.merge(df_res,left_index=True,right_index=True)

In [37]:
df_new.head()

,date,u_timestamp,price,target,timestamp,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days,text,text_polarity,text_subjectivity,text_posneg,text_new,VEC_abc,VEC_acceler,VEC_accept,VEC_access,VEC_accord,VEC_account,VEC_accus,VEC_acquir,VEC_acquisit,VEC_act,VEC_action,VEC_activ,VEC_ad,VEC_adam,VEC_add,VEC_address,VEC_adopt,VEC_advertis,VEC_affect,VEC_africa,VEC_african,VEC_agenc,VEC_agent,VEC_aggreg,VEC_ahead,VEC_aid,VEC_aim,VEC_air,...,VEC_watchdog,VEC_wave,VEC_way,VEC_wealth,VEC_web,VEC_websit,VEC_week,VEC_wendi,VEC_west,VEC_whale,VEC_white,VEC_wild,VEC_win,VEC_wirecard,VEC_wirex,VEC_withdraw,VEC_withdrawn,VEC_woman,VEC_wordpress,VEC_world,VEC_worldwid,VEC_worri,VEC_worst,VEC_worth,VEC_wrap,VEC_xrp,VEC_yam,VEC_year,VEC_yearold,VEC_yield,VEC_york,VEC_young,VEC_youtub,VEC_yuan,VEC_zcash,VEC_zebpay,VEC_zec,VEC_zero,VEC_zhuoer,VEC_zone
0,28-08-2020 17:00:00,1.598634e+09,11545.0,1,1577898000,3,['etoro ceo yoni assia on reaching 12 million ...,etoro ceo yoni assia on reaching 12 million us...,0.2023,0.000000,0.000000,etoro ceo yoni assia reach million user crypto...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,27-08-2020 17:00:00,1.598548e+09,11343.0,0,1577984400,7,['government confirms crypto profits not taxab...,government confirms crypto profits not taxable...,0.9136,0.462500,5.656250,govern confirm crypto profit taxabl south_kore...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,26-08-2020 17:00:00,1.598461e+09,11471.0,1,1578070800,8,['ticker tool uses the bch blockchain to provi...,ticker tool uses the bch blockchain to provide...,-0.5647,0.254167,0.762295,ticker tool use bch blockchain provid reliabl ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
3,25-08-2020 17:00:00,1.598375e+09,11341.0,0,1578157200,6,['crypto fundraising is changing again in 2020...,crypto fundraising is changing again in 2020 x...,0.1531,0.518182,1.428571,crypto fundrais chang proof key day return bit...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,23-08-2020 17:00:00,1.598202e+09,11663.0,0,1578243600,5,['subhash chandra garg on the future of crypto...,subhash chandra garg on the future of crypto x...,0.8126,0.319886,3.653061,subhash chandra garg futur crypto ross ulbrich...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [82]:
df_final = df_new[df_new.columns[~df_new.columns.isin(['date','timestamp','price','BITCOIN_NE.text_COUNT_last1days','BITCOIN_NE.text_last1days','text','VEC.source','text_new'])]]

In [ ]:
df_final.head()

,u_timestamp,target,text_polarity,text_subjectivity,text_posneg,VEC_abc,VEC_acceler,VEC_accept,VEC_access,VEC_accord,VEC_account,VEC_accus,VEC_acquir,VEC_acquisit,VEC_act,VEC_action,VEC_activ,VEC_ad,VEC_adam,VEC_add,VEC_address,VEC_adopt,VEC_advertis,VEC_affect,VEC_africa,VEC_african,VEC_agenc,VEC_agent,VEC_aggreg,VEC_ahead,VEC_aid,VEC_aim,VEC_air,VEC_airdrop,VEC_airport,VEC_algorithm,VEC_alleg,VEC_allow,VEC_alltim,VEC_altcoin,...,VEC_watchdog,VEC_wave,VEC_way,VEC_wealth,VEC_web,VEC_websit,VEC_week,VEC_wendi,VEC_west,VEC_whale,VEC_white,VEC_wild,VEC_win,VEC_wirecard,VEC_wirex,VEC_withdraw,VEC_withdrawn,VEC_woman,VEC_wordpress,VEC_world,VEC_worldwid,VEC_worri,VEC_worst,VEC_worth,VEC_wrap,VEC_xrp,VEC_yam,VEC_year,VEC_yearold,VEC_yield,VEC_york,VEC_young,VEC_youtub,VEC_yuan,VEC_zcash,VEC_zebpay,VEC_zec,VEC_zero,VEC_zhuoer,VEC_zone
0,1.598634e+09,1,0.2023,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.598548e+09,0,0.9136,0.462500,5.656250,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.598461e+09,1,-0.5647,0.254167,0.762295,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
3,1.598375e+09,0,0.1531,0.518182,1.428571,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.598202e+09,0,0.8126,0.319886,3.653061,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [83]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score# We drop correlated features because with so many binary 
# ngram variables there's a lot of noise

corr_matrix = df_final.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

In [40]:
corr_matrix

,u_timestamp,target,text_polarity,text_subjectivity,text_posneg,VEC_abc,VEC_acceler,VEC_accept,VEC_access,VEC_accord,VEC_account,VEC_accus,VEC_acquir,VEC_acquisit,VEC_act,VEC_action,VEC_activ,VEC_ad,VEC_adam,VEC_add,VEC_address,VEC_adopt,VEC_advertis,VEC_affect,VEC_africa,VEC_african,VEC_agenc,VEC_agent,VEC_aggreg,VEC_ahead,VEC_aid,VEC_aim,VEC_air,VEC_airdrop,VEC_airport,VEC_algorithm,VEC_alleg,VEC_allow,VEC_alltim,VEC_altcoin,...,VEC_watchdog,VEC_wave,VEC_way,VEC_wealth,VEC_web,VEC_websit,VEC_week,VEC_wendi,VEC_west,VEC_whale,VEC_white,VEC_wild,VEC_win,VEC_wirecard,VEC_wirex,VEC_withdraw,VEC_withdrawn,VEC_woman,VEC_wordpress,VEC_world,VEC_worldwid,VEC_worri,VEC_worst,VEC_worth,VEC_wrap,VEC_xrp,VEC_yam,VEC_year,VEC_yearold,VEC_yield,VEC_york,VEC_young,VEC_youtub,VEC_yuan,VEC_zcash,VEC_zebpay,VEC_zec,VEC_zero,VEC_zhuoer,VEC_zone
u_timestamp,1.000000,0.078198,0.129950,0.236463,0.167387,0.115901,0.066068,0.039389,0.143596,0.008230,0.000393,0.003665,0.019009,0.091059,0.020138,0.079987,0.043276,0.022020,0.015346,0.093893,0.163510,0.002701,0.084980,0.043839,0.144465,0.174544,0.126182,0.009581,0.048081,0.066975,0.019713,0.006368,0.004494,0.010375,0.110647,0.003992,0.021853,0.037789,0.005143,0.021481,...,0.112064,0.013368,0.056107,0.021431,0.032546,0.024921,0.023213,0.121712,0.002827,0.056622,0.110647,0.002640,0.096442,0.111939,0.125757,0.109572,0.060665,0.125757,0.037274,0.025097,0.051022,0.002886,0.018668,0.113708,0.053485,0.054067,0.141967,0.072999,0.085230,0.163742,0.119002,0.026217,0.048848,0.146294,0.010682,0.031626,0.053910,0.110636,0.122130,0.085235
target,0.078198,1.000000,0.003190,0.007029,0.052693,0.010044,0.085648,0.034943,0.055801,0.007072,0.098361,0.016091,0.055801,0.085648,0.085648,0.099785,0.081797,0.075889,0.099792,0.114605,0.068949,0.055801,0.007072,0.040764,0.029090,0.040764,0.085648,0.085648,0.085648,0.010044,0.007072,0.033772,0.075889,0.010044,0.099792,0.046614,0.111604,0.074666,0.040764,0.136298,...,0.010044,0.099792,0.099785,0.029254,0.099792,0.099785,0.033726,0.029254,0.007072,0.038423,0.007072,0.105122,0.029090,0.055801,0.099792,0.032637,0.085648,0.085648,0.085648,0.075870,0.055801,0.029254,0.029254,0.084338,0.085648,0.061775,0.007072,0.066050,0.007072,0.010044,0.007072,0.007072,0.126570,0.011676,0.007072,0.029254,0.007072,0.111874,0.085648,0.029254
text_polarity,0.129950,0.003190,1.000000,0.001274,0.583386,0.054351,0.058027,0.072051,0.006972,0.037018,0.085059,0.005516,0.010643,0.029212,0.047609,0.000997,0.036907,0.086448,0.036975,0.062010,0.010845,0.022510,0.033514,0.104821,0.046757,0.071274,0.031502,0.036637,0.024994,0.019639,0.069599,0.018852,0.028783,0.091986,0.038078,0.073862,0.030001,0.033917,0.103003,0.049285,...,0.023919,0.030591,0.112242,0.133677,0.041828,0.033117,0.191406,0.119321,0.034152,0.141605,0.059687,0.026526,0.215738,0.064722,0.116646,0.041653,0.015912,0.068764,0.079212,0.036023,0.088103,0.017771,0.156516,0.020595,0.007492,0.012149,0.018611,0.064015,0.095532,0.072907,0.021596,0.076067,0.005073,0.015551,0.012973,0.121982,0.004994,0.027135,0.049045,0.055477
text_subjectivity,0.236463,0.007029,0.001274,1.000000,0.073185,0.063398,0.042495,0.007697,0.060223,0.005638,0.004968,0.050605,0.003650,0.086683,0.036969,0.118932,0.051700,0.038725,0.015315,0.038848,0.007517,0.008891,0.060806,0.085339,0.007562,0.033534,0.050886,0.018532,0.038234,0.119511,0.034231,0.002037,0.157462,0.043102,0.125347,0.065077,0.040683,0.102110,0.020624,0.055990,...,0.067773,0.040068,0.023078,0.011863,0.040448,0.184690,0.065208,0.017602,0.038156,0.038765,0.027668,0.025561,0.027297,0.003412,0.061948,0.062764,0.012059,0.050169,0.055710,0.007816,0.005478,0.031602,0.151203,0.082274,0.001615,0.056855,0.002066,0.006338,0.010163,0.019045,0.001155,0.047588,0.024855,0.038332,0.006348,0.039281,0.062913,0.010224,0.026297,0.063931
text_posneg,0.167387,0.052693,0.583386,0.073185,1.000000,0.184209,0.061268,0.043146,0.016397,0.006954,0.021911,0.015647,0.013653,0.054681,0.030580,0.014715,0.010864,0.063475,0.020209,0.064504,0.025656,0.006614,0.0

In [84]:
df_final.drop([column for column in upper.columns if any(upper[column] > 0.5)], axis=1, inplace=True)
# Now we separate in train/test sets
df_final.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,u_timestamp,target,text_polarity,text_subjectivity,VEC_abc,VEC_acceler,VEC_accept,VEC_access,VEC_accord,VEC_account,VEC_accus,VEC_acquir,VEC_acquisit,VEC_act,VEC_action,VEC_activ,VEC_ad,VEC_adam,VEC_add,VEC_address,VEC_adopt,VEC_advertis,VEC_affect,VEC_africa,VEC_african,VEC_agenc,VEC_agent,VEC_aggreg,VEC_ahead,VEC_aid,VEC_aim,VEC_air,VEC_airdrop,VEC_airport,VEC_algorithm,VEC_alleg,VEC_allow,VEC_alltim,VEC_altcoin,VEC_altern,...,VEC_valu,VEC_valuat,VEC_viabtc,VEC_view,VEC_violat,VEC_viral,VEC_virtual,VEC_volum,VEC_vote,VEC_wall_street,VEC_wallet,VEC_wanna,VEC_warn,VEC_watch,VEC_watchdog,VEC_wealth,VEC_web,VEC_white,VEC_wild,VEC_win,VEC_wirex,VEC_withdraw,VEC_withdrawn,VEC_world,VEC_worldwid,VEC_worri,VEC_worst,VEC_worth,VEC_wrap,VEC_year,VEC_yearold,VEC_yield,VEC_york,VEC_youtub,VEC_yuan,VEC_zcash,VEC_zebpay,VEC_zec,VEC_zero,VEC_zone
0,1.598634e+09,1,0.2023,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.598548e+09,0,0.9136,0.462500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.598461e+09,1,-0.5647,0.254167,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3,1.598375e+09,0,0.1531,0.518182,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.598202e+09,0,0.8126,0.319886,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [42]:
df_final.head()

,u_timestamp,target,text_polarity,text_subjectivity,text_posneg,VEC_abc,VEC_acceler,VEC_accept,VEC_access,VEC_accord,VEC_account,VEC_accus,VEC_acquir,VEC_acquisit,VEC_act,VEC_action,VEC_activ,VEC_ad,VEC_adam,VEC_add,VEC_address,VEC_adopt,VEC_advertis,VEC_affect,VEC_africa,VEC_african,VEC_agenc,VEC_agent,VEC_aggreg,VEC_ahead,VEC_aid,VEC_aim,VEC_air,VEC_airdrop,VEC_airport,VEC_algorithm,VEC_alleg,VEC_allow,VEC_alltim,VEC_altcoin,...,VEC_wallet,VEC_wanna,VEC_want,VEC_war,VEC_warn,VEC_watch,VEC_watchdog,VEC_way,VEC_wealth,VEC_web,VEC_websit,VEC_week,VEC_west,VEC_whale,VEC_white,VEC_wild,VEC_win,VEC_wirecard,VEC_wirex,VEC_withdraw,VEC_withdrawn,VEC_world,VEC_worldwid,VEC_worri,VEC_worst,VEC_worth,VEC_wrap,VEC_yam,VEC_year,VEC_yearold,VEC_yield,VEC_york,VEC_young,VEC_youtub,VEC_yuan,VEC_zcash,VEC_zebpay,VEC_zec,VEC_zero,VEC_zone
0,1.598634e+09,1,0.2023,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.598548e+09,0,0.9136,0.462500,5.656250,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.598461e+09,1,-0.5647,0.254167,0.762295,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
3,1.598375e+09,0,0.1531,0.518182,1.428571,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.598202e+09,0,0.8126,0.319886,3.653061,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [98]:
df_final['senttiment_change'] = df_final['text_polarity'].diff(1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [100]:
df_final = df_final.iloc[1:,:]

In [101]:
df_final.head()

,u_timestamp,target,text_polarity,text_subjectivity,VEC_abc,VEC_acceler,VEC_accept,VEC_access,VEC_accord,VEC_account,VEC_accus,VEC_acquir,VEC_acquisit,VEC_act,VEC_action,VEC_activ,VEC_ad,VEC_adam,VEC_add,VEC_address,VEC_adopt,VEC_advertis,VEC_affect,VEC_africa,VEC_african,VEC_agenc,VEC_agent,VEC_aggreg,VEC_ahead,VEC_aid,VEC_aim,VEC_air,VEC_airdrop,VEC_airport,VEC_algorithm,VEC_alleg,VEC_allow,VEC_alltim,VEC_altcoin,VEC_altern,...,VEC_valuat,VEC_viabtc,VEC_view,VEC_violat,VEC_viral,VEC_virtual,VEC_volum,VEC_vote,VEC_wall_street,VEC_wallet,VEC_wanna,VEC_warn,VEC_watch,VEC_watchdog,VEC_wealth,VEC_web,VEC_white,VEC_wild,VEC_win,VEC_wirex,VEC_withdraw,VEC_withdrawn,VEC_world,VEC_worldwid,VEC_worri,VEC_worst,VEC_worth,VEC_wrap,VEC_year,VEC_yearold,VEC_yield,VEC_york,VEC_youtub,VEC_yuan,VEC_zcash,VEC_zebpay,VEC_zec,VEC_zero,VEC_zone,senttiment_change
1,1.598548e+09,0,0.9136,0.462500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.7113
2,1.598461e+09,1,-0.5647,0.254167,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,-1.4783
3,1.598375e+09,0,0.1531,0.518182,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.7178
4,1.598202e+09,0,0.8126,0.319886,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.6595
5,1.598116e+09,1,0.6908,0.700000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.1218


In [102]:
X = df_final.iloc[:,2:].values
y = df_final.iloc[:,1].values

In [103]:
div = int(round(len(X) * 0.2))
X_train = X[:div]
y_train = y[:div]
X_test = X[div:]
y_test = y[div:]
# Finally, we perform ML and see results

In [104]:
X_train[:10]

array([[ 0.9136    ,  0.4625    ,  0.        , ...,  0.        ,
         0.        ,  0.7113    ],
       [-0.5647    ,  0.25416667,  0.        , ...,  0.        ,
         0.        , -1.4783    ],
       [ 0.1531    ,  0.51818182,  0.        , ...,  0.        ,
         0.        ,  0.7178    ],
       ...,
       [ 0.2691    ,  0.48166667,  0.        , ...,  0.        ,
         0.        , -0.1897    ],
       [-0.7351    ,  0.70666667,  0.        , ...,  0.        ,
         0.        , -1.0042    ],
       [-0.483     ,  0.76493506,  0.        , ...,  0.        ,
         1.        ,  0.2521    ]])

In [105]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics


In [106]:
from scipy.stats import uniform, truncnorm, randint

In [107]:
model_params = {
    # randomly sample numbers from 4 to 204 estimators
    'n_estimators': randint(4,300),
    # normally distributed max_features, with mean .25 stddev 0.1, bounded between 0 and 1
    'max_features': truncnorm(a=0, b=1, loc=0.25, scale=0.1),
    # uniform distribution from 0.01 to 0.2 (0.01 + 0.199)
    'min_samples_split': uniform(0.01, 0.199)
}

In [108]:
forest = RandomForestClassifier(random_state = 0)

In [109]:
# rf = RandomForestRegressor(n_estimators = 1000, random_state=0)
# rf.fit(X_train, y_train)
# y_pred = rf.predict(X_test)
# df_res = pd.DataFrame({'y_test':y_test, 'y_pred':y_pred})

In [112]:
clf = RandomizedSearchCV(forest,model_params,n_iter=100,cv=5)

In [113]:
model = clf.fit(X_train, y_train)

In [114]:
from pprint import pprint
pprint(model.best_estimator_.get_params())

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 0.33733935465647247,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 0.03527009569233812,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 36,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}


In [116]:
model.best_score_

0.6599999999999999

In [126]:
y_pred_prob = np.max(model.predict_proba(X_test),axis=1)

In [132]:
threshold = 0.5
y_pred_new = [1 if val > threshold else 0 for val in y_pred_prob]

In [133]:
print(metrics.confusion_matrix(y_pred_new, y_test))
print('Accuracy Score:')
print(accuracy_score(y_pred_new, y_test))
print('Precision Score:')
print(precision_score(y_pred_new, y_test))

[[ 2  3]
 [89 94]]
Accuracy Score:
0.5106382978723404
Precision Score:
0.9690721649484536


### Test on new data


In [ ]:
### News API
news_api_key = "d54b0ca28da842519b0a2e796fd33932"
import pprint 
import requests 
  
  
secret = news_api_key
   
# Define the endpoint 
url = 'https://newsapi.org/v2/everything?'
   
# Specify the query and 
# number of returns 
parameters = { 
    'q': 'Bitcoin', # query phrase 
    'pageSize': 100,  # maximum is 100 
    'apiKey': secret # your own API key 
} 
   
# Make the request 
response = requests.get(url,  
                        params = parameters) 
   
# Convert the response to  
# JSON format and pretty print it 
response_json = response.json() 
pprint.pprint(response_json) 

{'articles': [{'author': 'Timothy B. Lee, Ars Technica',
               'content': 'A former Microsoft software engineer from Ukraine '
                          'has been sentenced to nine years in prison for '
                          'stealing more than $10 million in store credit from '
                          "Microsoft's online store. From 2016 to 2018, Volod… "
                          '[+3307 chars]',
               'description': 'The defendant tried—and failed—to use bitcoin '
                              'to cover his tracks.',
               'publishedAt': '2020-11-12T14:00:00Z',
               'source': {'id': 'wired', 'name': 'Wired'},
               'title': 'An Engineer Gets 9 Years for Stealing $10M From '
                        'Microsoft',
               'url': 'https://www.wired.com/story/an-engineer-gets-9-years-for-stealing-dollar10m-from-microsoft/',
               'urlToImage': 'https://media.wired.com/photos/5fac6afb446b4639b3d5b8d8/191:100/w_1280,c_limit

In [ ]:
def flatten_dict(d):
    """ Returns list of lists from given dictionary """
    l = []
    for k, v in sorted(d.items()):
        if isinstance(v, dict):
            flatten_v = flatten_dict(v)
            for my_l in reversed(flatten_v):
                my_l.insert(0, k)

            l.extend(flatten_v)

        elif isinstance(v, list):
            for l_val in v:
                l.append([k, l_val])

        else:
            l.append([k, v])

    return l

In [ ]:
df = pd.DataFrame(flatten_dict(response_json))

In [ ]:
df = pd.Series(flatten_json_iterative_solution(dict(json_))).to_frame().reset_index()

,0,1
0,articles,"{'source': {'id': 'wired', 'name': 'Wired'}, '..."
1,articles,"{'source': {'id': None, 'name': 'Lifehacker.co..."
2,articles,"{'source': {'id': 'engadget', 'name': 'Engadge..."
3,articles,"{'source': {'id': 'mashable', 'name': 'Mashabl..."
4,articles,"{'source': {'id': 'engadget', 'name': 'Engadge..."
...,...,...
97,articles,"{'source': {'id': None, 'name': 'Coincolony.ne..."
98,articles,"{'source': {'id': None, 'name': 'heise online'..."
99,articles,"{'source': {'id': 'business-insider', 'name': ..."
100,status,ok


In [ ]:
df = pd.DataFrame.from_dict(response_json)

In [ ]:
df['content'] = df['articles'].apply(lambda x:x['content'])
df['Date'] = df['articles'].apply(lambda x:x['publishedAt'])

In [ ]:
df.head()

,status,totalResults,articles,content,Date
0,ok,6573,"{'source': {'id': 'wired', 'name': 'Wired'}, '...",A former Microsoft software engineer from Ukra...,2020-11-12T14:00:00Z
1,ok,6573,"{'source': {'id': None, 'name': 'Lifehacker.co...",Visa has partnered with cryptocurrency startup...,2020-12-03T22:00:00Z
2,ok,6573,"{'source': {'id': 'engadget', 'name': 'Engadge...",PayPal is bringing its newly-announced support...,2020-11-12T21:05:41Z
3,ok,6573,"{'source': {'id': 'mashable', 'name': 'Mashabl...","In November 2017, after an absolutely massive,...",2020-11-20T20:02:17Z
4,ok,6573,"{'source': {'id': 'engadget', 'name': 'Engadge...","Unlike ‘conventional’ cryptocurrencies, a cent...",2020-12-06T22:37:18Z


In [ ]:
df['Date'] = df['Date'].apply(lambda x:pd.to_datetime(x[:10]))

In [ ]:
df = df.loc[:,['Date','content']]

In [ ]:
df

,Date,content
0,2020-11-12,A former Microsoft software engineer from Ukra...
1,2020-12-03,Visa has partnered with cryptocurrency startup...
2,2020-11-12,PayPal is bringing its newly-announced support...
3,2020-11-20,"In November 2017, after an absolutely massive,..."
4,2020-12-06,"Unlike ‘conventional’ cryptocurrencies, a cent..."
...,...,...
95,2020-12-01,* New Zealand dollar set to reach more than tw...
96,2020-12-01,(Fixes typo in the euro price in the 11th para...
97,2020-11-20,Its been nearly three years since the Bitcoin ...
98,2020-11-16,Bei der Bitcoin-Abspaltung Bitcoin Cash ist am...


In [ ]:
df.dropna(inplace=True)

In [ ]:
#dfTest = df.groupby(['Date']).agg(['concat'])
df.groupby(['Date'], as_index = False).agg({'content': ' '.join})

,Date,content
0,2020-11-12,A former Microsoft software engineer from Ukra...
1,2020-11-13,"November\r\n13, 2020\r\n5 min read\r\nOpinions..."
2,2020-11-16,Bei der Bitcoin-Abspaltung Bitcoin Cash ist am...
3,2020-11-17,"Maisie Williams also known as Arya Stark, daug..."
4,2020-11-18,8 with 7 posters participating\r\nBitcoin rose...
5,2020-11-19,LONDON (Reuters) - Is it an inflation hedge? A...
6,2020-11-20,"In November 2017, after an absolutely massive,..."
7,2020-11-23,FILE PHOTO: Representation of the Ethereum vir...
8,2020-11-24,A representation of virtual currency Bitcoin i...
9,2020-11-25,If youve been watching the crypto markets over...
